<a href="https://colab.research.google.com/github/flora0110/sentence_prediction/blob/main/bge_m3_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json

with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/dataset_new/train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/dataset_new/test_data.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [3]:
train_fact_len = []
test_fact_len = []

for data in train_data:
    train_fact_len.append(len(data['fact']))

for data in test_data:
    test_fact_len.append(len(data['fact']))


In [4]:
print("===Train Dataset===")
print("Length: ", len(train_fact_len))
print("Max Length: ", max(train_fact_len))
print("Min Length: ", min(train_fact_len))
print("Avg. Length: ", sum(train_fact_len) / len(train_fact_len))

print("===Test Dataset===")
print("Length: ", len(test_fact_len))
print("Max Length: ", max(test_fact_len))
print("Min Length: ", min(test_fact_len))
print("Avg. Length: ", sum(test_fact_len) / len(test_fact_len))

===Train Dataset===
Length:  500
Max Length:  19745
Min Length:  328
Avg. Length:  7933.656
===Test Dataset===
Length:  100
Max Length:  19277
Min Length:  280
Avg. Length:  8231.62


In [5]:
def extract_key_contents(data):
    facts = []
    sentences = []
    for item in data:
        facts.append(item.get('fact'))
        sentences.append(item.get('sentance'))
    return facts, sentences

In [6]:
train_facts, train_sentences = extract_key_contents(train_data)
test_facts, test_sentences = extract_key_contents(test_data)

In [7]:
print("Train facts:", train_facts[0])
print("Train Sentences:", train_sentences[:5])
print("Test facts:", test_facts[0])
print("Test Sentences:", test_sentences[:5])

Train facts: 一、陳泰龍明知其與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM，陳玉婉涉嫌本案偽造文書部分，另經臺灣臺南地方法院檢察署檢察官通緝中）間並無結婚之真意，然因詹巧薇、張晉源等人欲找臺灣男子前往泰國與泰國女子辦理假結婚，林清水負責介紹臺灣男子予詹巧薇、張晉源等人以賺取新臺幣（下同）1萬元之仲介費，適陳泰龍依林清水於報紙上刊登之徵人廣告應徵時，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬，迨陳泰龍應允同意後，林清水將其轉介紹予詹巧薇、張晉源等人，陳泰龍遂與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，先由陳泰龍依指示於民國93年4月3日前往泰國，並於同年月7日與陳玉婉在泰國曼谷市法拉坎農區註冊辦事處登記結婚，取得上開註冊辦事處核發之結婚登記書、結婚證書等文件後，陳泰龍即行返臺，並於同年5月19日持經我國駐泰國臺北經濟文化辦事處認證之上開結婚登記書、結婚證書等資料，至臺南縣永康市（改制前稱謂，以下沿用舊制稱謂）戶政事務所填寫結婚登記申請書，申請辦理其與陳玉婉之結婚登記，因此使上開戶政事務所僅有形式審查權之承辦公務員陷於錯誤，於形式上審核後，將陳泰龍與陳玉婉於93年4月7日結婚之不實事項，登載於其職務上所掌屬公文書之戶籍登記資料內，並據以核發戶籍謄本，而足以生損害於戶政機關對於戶政管理及戶籍登記資料維護之正確性。迨陳玉婉來臺後，為取得居留身分，再由陳泰龍、陳玉婉承前概括之犯意聯絡，於同年6月14日及94年5月5日，均持前開登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期而連續行使之（業經檢察官當庭補充），經承辦人員實質審查結果，均未發覺陳泰龍與陳玉婉假結婚之實情，而發給陳玉婉居留證或核准延期，各足以生損害於主管機關對外僑居留管理之正確性。二、案經臺南市警察局（現改制為臺南市政府警察局，以下沿用舊制稱謂）第一分局報告臺灣臺南地方法院檢察署檢察官偵查起訴。理由甲、程序方面：一、本件以下所引用具傳聞證據性質之供述證據，因本件被告陳泰龍、檢察官等人於本院準備程序進行時，均未爭執前開陳述之證據能力，且迄至本案言詞辯論終結前亦均未聲明異議，應視為已同意援引作為證據，又本院

In [8]:
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/test_summaries_sentences.json', 'r', encoding='utf-8') as f:
    test_summaries_sentences = json.load(f)

In [9]:
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/train_summaries_sentences_100.json', 'r', encoding='utf-8') as f:
    train_summaries_sentences_100 = json.load(f)
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/train_summaries_sentences_200.json', 'r', encoding='utf-8') as f:
    train_summaries_sentences_200 = json.load(f)
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/train_summaries_sentences_300.json', 'r', encoding='utf-8') as f:
    train_summaries_sentences_300 = json.load(f)
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/train_summaries_sentences_400.json', 'r', encoding='utf-8') as f:
    train_summaries_sentences_400 = json.load(f)
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/train_summaries_sentences_500.json', 'r', encoding='utf-8') as f:
    train_summaries_sentences_500 = json.load(f)

# 連接summary


In [15]:
print(len(test_summaries_sentences))
print(len(test_summaries_sentences[0]))
print(test_summaries_sentences[0])

100
3
['因徐鴻光需款使用，知悉穆立明有借款管道，為求順利借款，2人竟共同意圖為自己不法所有，基於行使偽造私文書及詐欺取財之犯意聯絡，偽造「陳健志」代表志朗公司與徐鴻光簽署的「臺北市計程車客運駕駛人自備車輛參與經營契約書」，並持上開偽刻印章，偽造「志朗交通有限公司」印文4枚、「陳健志」印文6枚於本案參與經營契約書上，再於92年3月6日交徐鴻光於立約人乙方欄簽名。', '嗣因徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，王傑生始查知上開車輛非徐鴻光所有，而悉上情。', '二、案經王傑生訴由臺灣士林地方法院檢察署檢察官偵查起訴。']


In [10]:
test_summaries = []
for summary in test_summaries_sentences:
  text = ""
  for sentence in summary:
    text = text + sentence
  test_summaries.append(text)

In [16]:
print(len(test_summaries))
print(test_summaries[0])

100
因徐鴻光需款使用，知悉穆立明有借款管道，為求順利借款，2人竟共同意圖為自己不法所有，基於行使偽造私文書及詐欺取財之犯意聯絡，偽造「陳健志」代表志朗公司與徐鴻光簽署的「臺北市計程車客運駕駛人自備車輛參與經營契約書」，並持上開偽刻印章，偽造「志朗交通有限公司」印文4枚、「陳健志」印文6枚於本案參與經營契約書上，再於92年3月6日交徐鴻光於立約人乙方欄簽名。嗣因徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，王傑生始查知上開車輛非徐鴻光所有，而悉上情。二、案經王傑生訴由臺灣士林地方法院檢察署檢察官偵查起訴。


In [19]:
print(len(train_summaries_sentences_100))
print(len(train_summaries_sentences_100[0]))
print(len(train_summaries_sentences_200))
print(len(train_summaries_sentences_200[0]))
print(len(train_summaries_sentences_300))
print(len(train_summaries_sentences_300[0]))
print(len(train_summaries_sentences_400))
print(len(train_summaries_sentences_400[0]))
print(len(train_summaries_sentences_500))
print(len(train_summaries_sentences_500[0]))

100
3
100
3
100
3
100
13
100
3


In [11]:
train_summaries = []
for summary in train_summaries_sentences_100:
  text = ""
  for sentence in summary:
    text = text + sentence
  train_summaries.append(text)

In [12]:
for summary in train_summaries_sentences_200:
  text = ""
  for sentence in summary:
    text = text + sentence
  train_summaries.append(text)

In [13]:
for summary in train_summaries_sentences_300:
  text = ""
  for sentence in summary:
    text = text + sentence
  train_summaries.append(text)

In [14]:
for summary in train_summaries_sentences_400:
  text = ""
  for sentence in summary:
    text = text + sentence
  train_summaries.append(text)

In [15]:
for summary in train_summaries_sentences_500:
  text = ""
  for sentence in summary:
    text = text + sentence
  train_summaries.append(text)

In [16]:
train_summaries_len = []
test_summaries_len = []

for data in train_summaries:
    train_summaries_len.append(len(data))

for data in test_summaries:
    test_summaries_len.append(len(data))

In [17]:
print("===Train Dataset===")
print("Length: ", len(train_summaries_len))
print("Max Length: ", max(train_summaries_len))
print("Min Length: ", min(train_summaries_len))
print("Avg. Length: ", sum(train_summaries_len) / len(train_summaries_len))

print("===Test Dataset===")
print("Length: ", len(test_summaries_len))
print("Max Length: ", max(test_summaries_len))
print("Min Length: ", min(test_summaries_len))
print("Avg. Length: ", sum(test_summaries_len) / len(test_summaries_len))

===Train Dataset===
Length:  500
Max Length:  1291
Min Length:  0
Avg. Length:  388.652
===Test Dataset===
Length:  100
Max Length:  1219
Min Length:  97
Avg. Length:  394.01


# 斷詞

In [49]:
!pip install udicOpenData

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for udicOpenData: filename=udicOpenData-2.4-py3-none-any.whl size=18094423 sha256=dc95b9e256633d7fc67dd7b40052874a587f27d0c4ffc4ebab4546775b06e857
  Stored in directory: /root/.cache/pip/wheels/86/4f/46/2a28bc28485ba1f9d67dead62ac6aba14089a6cf1d5fbad365
Successfully built udicOpenData


In [50]:
!pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 3.9 MB/s eta 0:00:00


In [51]:
from opencc import OpenCC

In [52]:
cc = OpenCC('t2s')

In [53]:
from udicOpenData.stopwords import *

# 分詞加空白
def split_passage(passage):
    word_and_tag = list(rmsw(passage, flag=True))
    _temp = list(filter(lambda x: x[1]!='nr' and x[1]!='ns', word_and_tag))
    # print(_temp)
    text = " ".join(list(map(lambda x:x[0], _temp)))
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [54]:
tokenized_train_summaries = []

for data in train_summaries:
    tokenized_train_summaries.append(split_passage(cc.convert(data)))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.977 seconds.
DEBUG:jieba:Loading model cost 0.977 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [55]:
tokenized_test_summaries = []

for data in test_summaries:
    tokenized_test_summaries.append(split_passage(cc.convert(data)))

In [56]:
def save_string_array_to_file(string_array, file_path):
    try:
        with open(file_path, 'w') as file:
            for string in string_array:
                file.write(string + '\n')
        print(f"String array saved to {file_path} successfully.")
    except Exception as e:
        print(f"Error occurred while saving string array: {e}")

In [57]:
# save_string_array_to_file(tokenized_train_summaries, '/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt')
# save_string_array_to_file(tokenized_test_summaries, '/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt')

String array saved to /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt successfully.
String array saved to /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt successfully.


In [18]:
def read_string_array_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            string_array = file.readlines()
            # 去除每個元素末尾的換行符
            string_array = [string.strip() for string in string_array]
        print(f"String array read from {file_path} successfully.")
        return string_array
    except Exception as e:
        print(f"Error occurred while reading string array: {e}")
        return None

In [21]:
tokenized_train_summaries = read_string_array_from_file('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt')
tokenized_test_summaries = read_string_array_from_file('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt')

String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_train_summaries.txt successfully.
String array read from /content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/tokenized_test_summaries.txt successfully.


In [22]:
print(tokenized_train_summaries[0])
print(tokenized_train_summaries[1])
print(tokenized_test_summaries[0])
print(tokenized_test_summaries[1])

被告 明知 与 籍 女子 JARUWANWONGPRATOOM 间 并 无 结婚 真意 与 清水 源等 人 共同 基于 行使 使 公务员 登载 不实 公文书 概括 犯意 联络 办理 假结婚 手续 被告 与 办理 结婚 手续 后 返回 办理 结婚登记 使 承办 公务员 陷于 错误 登载 不实 事项 于 户籍 登记 资料 内 核发 户籍誊本 来 台后 为 取得 居留 身分 被告 与 共同 县政府 警察局 外事 课 申请 核发 延期 承办 人员 实质 审查 结果 未 发觉 假结婚 核发 核准 延期
甲 于 民国 上午 时许 北安 桥 路旁 拾得 发射 子弹 具有 杀伤力 枪枝 枪枝 管制 编号 号 后 持有 并 于 主动 员警 坦承 犯行 自首 接受 裁判 甲 于 本院 审理 该案 年度 诉 字 号 时 编造 母 于 经 急诊 住院 接受 治疗 系 为 照顾 住院 母亲 报缴 枪枝 情节 辩称 无 持有 发射 子弹 具有 杀伤力 枪枝 犯 意甲 为 取信于 法院 基于 行使 伪造 私 文书 犯意 于 间 某日 不详 地点 伪刻 奇美 医疗 财团法人 奇美 医院 下称 奇美 医院 院长 医师 甲 印章 颗 伪造 附表 编号 所示 诊断 证明书 下称 系争 诊断 证明书 提出 于 本院 收件 日期 为 表示 系争 诊断 证明书 记载 内容 业经 认证 属实 意 足以 生 损害 于 奇美 医院 甲 国家 行使 正确性
需款 使用 知悉 借款 管道 为求 顺利 借款 人 共同 意图 为 不法 所有 基于 行使 伪造 私 文书 诈欺 取财 犯意 联络 伪造 代表 公司 与 签署 计程车 客运 驾驶 人 自备 车辆 参与 经营 契约书 并持 上开 伪刻 印章 伪造 交通 有限公司 印文 枚 印文 枚 于 本案 参与 经营 契约书 于 交 约人 乙方 栏 签名 嗣 仅 清偿 每期 千元 分期 款 期计 万 千元 未 全额 清偿 无力 负担 债务 生始 查知 上开 车辆 非 所有 悉上 情 案经 生诉 士林 地方法院 检察署 检察官 侦查 起诉
主动 与 祭祀 公业 子玉 管理 人 赐福 耀西 联络 处理 该 祭祀 公业 相关 事宜 经 洽商 结果 于 埙 三等 人 需 支费 下 代为 申办 事成后 获得 该 祭祀 公业 所有 土地 价值 报酬 与 埙 共同 基于 意图 为 自已 不法 

In [23]:
tokenized_train_summaries_len = []
tokenized_test_summaries_len = []

for data in tokenized_train_summaries:
    tokenized_train_summaries_len.append(len(data))

for data in tokenized_test_summaries:
    tokenized_test_summaries_len.append(len(data))

In [24]:
print("===Train Dataset===")
print("Length: ", len(tokenized_train_summaries_len))
print("Max Length: ", max(tokenized_train_summaries_len))
print("Min Length: ", min(tokenized_train_summaries_len))
print("Avg. Length: ", sum(tokenized_train_summaries_len) / len(tokenized_train_summaries_len))

print("===Test Dataset===")
print("Length: ", len(tokenized_test_summaries_len))
print("Max Length: ", max(tokenized_test_summaries_len))
print("Min Length: ", min(tokenized_test_summaries_len))
print("Avg. Length: ", sum(tokenized_test_summaries_len) / len(tokenized_test_summaries_len))

===Train Dataset===
Length:  500
Max Length:  1358
Min Length:  0
Avg. Length:  389.714
===Test Dataset===
Length:  100
Max Length:  1219
Min Length:  91
Avg. Length:  394.76


# bge-m3

In [60]:
!pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.

## 引入模型並測試在中文上的效果

In [61]:
from FlagEmbedding import BGEM3FlagModel

In [62]:
model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

colbert_linear.pt:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

In [63]:
sword_1 = ["老师", "商品"]
sword_2 = ["教师", "货物"]

## dense

### 測試在中文上的效果

In [64]:
embeddings_1 = model.encode(sword_1,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sword_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

[[0.8509769  0.5041799 ]
 [0.464958   0.74887514]]


### train test data embedding

In [ ]:
train_embeddings = model.encode(tokenized_train_summaries, max_length=1500)['dense_vecs']

Inference Embeddings:  81%|████████  | 34/42 [1:20:17<19:33, 146.66s/it]

In [ ]:
test_embeddings = model.encode(tokenized_test_summaries, max_length=1500)['dense_vecs']

In [ ]:
import numpy as np
np.savetxt('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/train_embeddings.txt', train_embeddings)
np.savetxt('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/test_embeddings.txt', test_embeddings)

In [ ]:
# loaded_train_embeddings = np.loadtxt('train_embeddings.txt')

In [107]:
type(train_embeddings)

numpy.ndarray

In [128]:
train_embeddings

array([[-0.03034765,  0.02622485, -0.01174086, ..., -0.01362408,
        -0.04923247, -0.0444256 ],
       [-0.02140026,  0.0173902 , -0.06489349, ..., -0.00908999,
        -0.02669849,  0.02307938],
       [-0.01579434,  0.04073653, -0.02186119, ..., -0.02366436,
        -0.0657978 , -0.05442155],
       ...,
       [-0.05367578,  0.01930024, -0.00059708, ...,  0.015949  ,
        -0.05102739,  0.01105056],
       [-0.01177966,  0.02777622, -0.02606803, ..., -0.01549191,
        -0.04460454, -0.01327729],
       [-0.0269404 ,  0.02274233, -0.03002858, ...,  0.02050726,
        -0.04272795,  0.0081914 ]], dtype=float32)

### 初始化KNN模型，指定使用Cosine Similarity計算距離

In [71]:
from sklearn.neighbors import NearestNeighbors

In [85]:
k = 5
knn_model_SBERT = NearestNeighbors(n_neighbors=k, metric='cosine')
knn_model_SBERT.fit(train_embeddings)

NearestNeighbors(metric='cosine')

In [86]:
import pickle

# 將模型保存到文件
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/knn_model_SBERT.pkl', 'wb') as f:
    pickle.dump(knn_model_SBERT, f)

# 從文件加載模型
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/knn_model_SBERT.pkl', 'rb') as f:
    knn_model_SBERT_loaded = pickle.load(f)

### 以第一篇test fact進行測試

In [87]:
distances, indices = knn_model_SBERT_loaded.kneighbors(test_embeddings[0].reshape(1, -1))

In [88]:
indices

array([[415,  47, 114, 250, 137]])

In [89]:
distances

array([[0.22361922, 0.26161647, 0.27732217, 0.28204942, 0.2864439 ]],
      dtype=float32)

In [90]:
test_facts[0]

'○、緣徐鴻光（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決處有期徒刑3月、緩刑3年確定）因需款使用，知悉穆立明有借款管道，為求順利借款，2人竟共同意圖為自己不法所有，基於行使偽造私文書及詐欺取財之犯意聯絡，知悉車牌號碼00-000號營業小客車係徐鴻光向志朗交通有限公司（下稱志朗公司）租賃之車輛，竟由穆立明先於不詳時間、不詳地點，偽刻「志朗交通有限公司」及負責人「陳健志」印章（正確姓名為陳建志，誤刻為「陳健志」）各1枚，並偽造「陳健志」代表志朗公司與徐鴻光簽署之「臺北市計程車客運駕駛人自備車輛參與經營契約書」（下稱本案參與經營契約書），虛偽記載徐鴻光提供上述營業小客車靠行志朗公司營業等不實內容，並持上開偽刻印章，偽造「志朗交通有限公司」印文4枚、「陳健志」印文6枚於本案參與經營契約書上，再於92年3月6日交徐鴻光於立約人乙方欄簽名，2人復於同日協同不知情之徐鴻光配偶徐樂仁（所涉行使偽造私文書犯行，業經本院以95年度訴字第955號判決無罪確定），前往臺北市○○區○○街00巷0號2樓王傑生所經營之宏帝當舖（現更名為渣打當舖），由徐鴻光、穆立明交付偽造本案參與經營契約書向宏帝當鋪借款新臺幣（下同）10萬元以為行使，並約定由徐樂仁擔任連帶保證人，王傑生誤信上開車輛為徐鴻光所有而陷入錯誤，於扣除利息後交付9萬1千元借款予徐鴻光夫婦，足以生損害於志朗公司、陳建志、王傑生。嗣因徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，王傑生始查知上開車輛非徐鴻光所有，而悉上情。二、案經王傑生訴由臺灣士林地方法院檢察署檢察官偵查起訴。理由○、證據能力部分：被告以外之人於審判外之言詞或書面陳述，除法律有規定者外，不得作為證據；又被告以外之人於審判外之陳述，雖不符合同法第159條之1至第159條之4之規定，但經當事人於審判程序同意作為證據，法院審酌該言詞陳述或書面陳述作成時之情況，認為適當者，亦得為證據。刑事訴訟法第159條第1項及第159條之5第1項分別定有明文。經查，本判決所引用之各該被告以外之人於審判外之陳述，雖屬傳聞證據，然當事人於本院審判期日中均表示同意作為證據方法而不予爭執，且迄至言詞辯論終結前亦未再聲明異議（見本院卷第68頁至第68頁背面），本院審酌上開證據資料製作時之情況，尚無違法不當及證明力明顯過低之瑕疵，亦認為以之作為證據應屬適當

In [91]:
test_summaries[0]

'因徐鴻光需款使用，知悉穆立明有借款管道，為求順利借款，2人竟共同意圖為自己不法所有，基於行使偽造私文書及詐欺取財之犯意聯絡，偽造「陳健志」代表志朗公司與徐鴻光簽署的「臺北市計程車客運駕駛人自備車輛參與經營契約書」，並持上開偽刻印章，偽造「志朗交通有限公司」印文4枚、「陳健志」印文6枚於本案參與經營契約書上，再於92年3月6日交徐鴻光於立約人乙方欄簽名。嗣因徐鴻光僅清償每期7千元之分期款共11期計7萬7千元，未全額清償即無力負擔債務，王傑生始查知上開車輛非徐鴻光所有，而悉上情。二、案經王傑生訴由臺灣士林地方法院檢察署檢察官偵查起訴。'

In [92]:
# 打印最相近的五篇文章的索引及距離
for i, index in enumerate(indices[0]):
    print("Article Index:", index, "Distance:", distances[0][i])
    print("Article Text:", train_facts[index])

Article Index: 415 Distance: 0.22361922
Article Text: 一、緣陳榮煉於民國106年7月間，借用李美娟之名義，以分期付款方式購入車牌號碼000-0000號自小客車後，僅繳納16期分期價金後，即因經濟困難而無力繳清餘款，而導致李美娟屢遭銀行催繳遲延價金（所涉詐欺罪嫌另由檢察官為不起訴處分），李美娟遂要求陳榮煉儘速將車輛讓渡，且不得再以其名義及該車輛借款。嗣陳榮煉復欲向第三人借款新臺幣(下同)14萬元，竟未得李美娟同意，基於偽造文書之犯意，於106年12月20日某時，在不詳地點偽造「李美娟」之署名1枚填載於「讓渡證書」之「立讓渡書人」欄位上，連同行車執照及本票2紙交由該第三人後行使之以作為其借款擔保，足以生損害於李美娟之權益。二、案經李美娟訴請臺灣嘉義地方檢察署檢察官偵查起訴。理由壹、程序部分：一、檢察官及被告陳榮煉對於下述本院採為認定犯罪事實依據之各項傳聞證據之證據能力，均表示同意有證據能力（見本院卷第31頁、第47頁、第48頁），另於言詞辯論終結前，除經同意有證據能力之證據外，對於卷附各傳聞證據之證據能力，均未聲明異議，復經本院於審理時逐一提示予被告表示意見，本院審酌該等證據之取得過程並無瑕疵，與本案待證事實間復具有相當之關聯性，以之為本案證據並無不當，依刑事訴訟法第159條之5規定自得採為本件認定事實之基礎。二、另其餘本判決所採之非供述證據亦均經法定程序取得，無不得為證據之情形，自均有證據能力。貳、實體部分：一、訊據被告陳榮煉固坦承因欲向案外人林采晴借款，在未經證人李美娟同意就以證人李美娟的名義簽立讓渡書，並將讓渡書交給案外人林采晴，讓渡書是案外人林采晴要求的，我當時是認為自己如果有還錢後這個讓渡書就無效，而且是證人李美娟一直說要叫我將車處理，所以我就簽了該讓渡書，我認為沒有造成損害等語（見本院卷第31頁、第32頁）。經查：(一)被告陳榮煉確未徵得證人李美娟同意即偽造「李美娟」之署名1枚填載於「讓渡證書」之「立讓渡書人」欄位，並將讓渡書作為向案外人林采晴借款之擔保一情，業據被告坦承不諱，核與證人即告訴人李美娟於偵查中以書面及本院證稱相符（見他字卷第58頁，本院卷第46頁），此外復有讓渡書1紙在卷可稽（見他字卷第101頁、第102頁），此部分事實應可認定。(二)又刑法上偽造文書罪之成立，以足生損害於公眾或他人為特別要

### 使用全部test facts在train facts中尋找最相近的五篇facts並計算預測刑期

In [93]:
predicted_sentences_all = []

for test_embedding in test_embeddings:
    test_embedding = test_embedding.reshape(1, -1)
    distances, indices = knn_model_SBERT_loaded.kneighbors(test_embedding)
    # 取得最相近的五篇文章的索引及距離
    nearest_articles_indices = indices[0]
    nearest_articles_distances = distances[0]
    predicted_sentences = 0
    # 將最相近的五篇文章的刑期加總，並計算平均值作為預測刑期
    for i, index in enumerate(nearest_articles_indices):
        predicted_sentences += train_sentences[index]
    predicted_sentence = float(predicted_sentences) / 5

    predicted_sentences_all.append(predicted_sentence)



In [94]:
print(predicted_sentences_all)
print(test_sentences)

[111.0, 102.0, 144.0, 100.0, 162.0, 308.0, 180.0, 265.0, 156.0, 211.0, 186.0, 193.0, 132.0, 126.0, 214.0, 138.0, 289.0, 150.0, 203.0, 185.0, 205.0, 138.0, 314.0, 149.0, 100.0, 107.0, 180.4, 136.0, 138.0, 97.6, 126.0, 121.0, 214.6, 211.0, 156.0, 139.6, 113.0, 240.0, 138.0, 138.0, 100.0, 231.0, 144.0, 241.0, 162.0, 235.0, 332.0, 114.0, 88.0, 186.0, 313.2, 108.0, 173.0, 211.0, 93.0, 151.0, 365.2, 126.0, 174.4, 176.0, 99.0, 148.0, 105.0, 108.0, 136.0, 170.0, 290.0, 150.0, 202.0, 156.0, 116.0, 141.0, 156.0, 175.0, 84.0, 283.4, 150.0, 170.0, 126.0, 153.0, 166.0, 95.4, 156.0, 301.0, 227.0, 165.0, 102.0, 165.0, 187.0, 162.0, 175.0, 132.0, 101.0, 356.0, 187.0, 402.2, 174.0, 120.0, 146.0, 125.0]
[150, 242, 60, 302, 365, 150, 90, 120, 120, 180, 120, 90, 120, 90, 90, 90, 180, 270, 425, 50, 150, 90, 365, 195, 150, 50, 345, 150, 150, 90, 425, 135, 120, 240, 20, 240, 60, 120, 60, 150, 90, 240, 365, 210, 150, 120, 32, 150, 505, 90, 90, 120, 120, 120, 165, 180, 302, 90, 120, 730, 240, 40, 180, 90, 90, 

In [95]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [96]:
mse_linear = mean_squared_error(test_sentences, predicted_sentences_all)
rmse_linear = mean_squared_error(test_sentences, predicted_sentences_all, squared=False)
mae_linear = mean_absolute_error(test_sentences, predicted_sentences_all)

In [97]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 21656.706400000003
Linear Regression - RMSE: 147.16217720596555
Linear Regression - MAE: 98.25200000000001


## sparse

In [ ]:
output_1 = model.encode(sentences_1, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(sentences_2, return_dense=True, return_sparse=True, return_colbert_vecs=False)

# you can see the weight for each token:
print(model.convert_id_to_token(output_1['lexical_weights']))
# [{'What': 0.08356, 'is': 0.0814, 'B': 0.1296, 'GE': 0.252, 'M': 0.1702, '3': 0.2695, '?': 0.04092},
#  {'De': 0.05005, 'fin': 0.1368, 'ation': 0.04498, 'of': 0.0633, 'BM': 0.2515, '25': 0.3335}]


# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)
# 0.19554901123046875

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))
# 0.0

[{'What': 0.08362087, 'is': 0.08146954, 'B': 0.12964633, 'GE': 0.25186998, 'M': 0.1700173, '3': 0.26957878, '?': 0.040755123}, {'De': 0.050144203, 'fin': 0.13689385, 'ation': 0.045134626, 'of': 0.06342198, 'BM': 0.25167608, '25': 0.33353215}]
0.19554448779672384
0


In [ ]:
output_1 = model.encode(tword_1, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(tword_2, return_dense=True, return_sparse=True, return_colbert_vecs=False)

# you can see the weight for each token:
print(model.convert_id_to_token(output_1['lexical_weights']))
# [{'What': 0.08356, 'is': 0.0814, 'B': 0.1296, 'GE': 0.252, 'M': 0.1702, '3': 0.2695, '?': 0.04092},
#  {'De': 0.05005, 'fin': 0.1368, 'ation': 0.04498, 'of': 0.0633, 'BM': 0.2515, '25': 0.3335}]


# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)
# 0.19554901123046875

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))
# 0.0

[{'': 0.06506694, '老師': 0.3252026}, {'': 0.025284417, '商品': 0.3079706}]
0.004433418158441782
0.0016451796982437372


In [ ]:
output_1 = model.encode(sword_1, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(sword_2, return_dense=True, return_sparse=True, return_colbert_vecs=False)

# you can see the weight for each token:
print(model.convert_id_to_token(output_1['lexical_weights']))
# [{'What': 0.08356, 'is': 0.0814, 'B': 0.1296, 'GE': 0.252, 'M': 0.1702, '3': 0.2695, '?': 0.04092},
#  {'De': 0.05005, 'fin': 0.1368, 'ation': 0.04498, 'of': 0.0633, 'BM': 0.2515, '25': 0.3335}]


# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)
# 0.19554901123046875

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))
# 0.0

[{'': 0.062333174, '老师': 0.3212393}, {'': 0.025284417, '商品': 0.3079706}]
0.0033142215106636286
0.0015760579844936728


In [ ]:
output_1 = model.encode(sample_1, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(sample_2, return_dense=True, return_sparse=True, return_colbert_vecs=False)

print(model.convert_id_to_token(output_1['lexical_weights']))

# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))

[{'陳': 0.2521531, '泰': 0.26584157, '龍': 0.30666173, '明': 0.101543695, '知': 0.12975708, '與': 0.14477229, '泰國': 0.22403938, '籍': 0.15496597, '女子': 0.20520654, '玉': 0.18034828, '婉': 0.29982036, '名字': 0.22417226, '為': 0.11855908, 'JAR': 0.15914416, 'U': 0.13418508, 'WAN': 0.17463517, 'W': 0.10995911, 'ONG': 0.16131966, 'P': 0.064036936, 'RAT': 0.15323746, 'OOM': 0.2604087, '涉嫌': 0.13823399, '本': 0.028688367, '案': 0.15687248, '偽': 0.22356482, '造': 0.1684515, '文書': 0.22656986, '部分': 0.04591082, ',': 0.028518757, '另': 0.012671046, '經': 0.02800768, '臺灣': 0.17963552, '臺': 0.20389023, '南': 0.103920385, '地方': 0.008341521, '法院': 0.055076532, '檢察': 0.06869293, '署': 0.028031692, '官': 0.028371364, '通': 0.014324367, '緝': 0.21278569, '中': 0.027402855, ')': 0.02840618, '間': 0.049174845, '並': 0.06908343, '無': 0.14975798, '結婚': 0.25648907, '之': 0.028479552, '真': 0.07777626, '意': 0.12144834, '然': 0.035248004, '因': 0.014868546, '詹': 0.19410798, '巧': 0.1502397, '薇': 0.24541272, '、': 0.028916981, '張': 0.14631

In [ ]:
lexical_scores1 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
lexical_scores2 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][1])

lexical_scores3 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][0])
lexical_scores4 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][1])

print(f'{lexical_scores1} {lexical_scores2}')
print(f'{lexical_scores3} {lexical_scores4}')

0.42963066551601514 0.0953057055012323
0.09472329181153327 0.28883391610725084


In [ ]:
output_1 = model.encode(tokenized_sample_1, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(tokenized_sample_2, return_dense=True, return_sparse=True, return_colbert_vecs=False)

print(model.convert_id_to_token(output_1['lexical_weights']))

# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))

[{'陳': 0.23019531, '泰': 0.25232542, '': 0.022680748, '龍': 0.23932248, '明': 0.10540678, '知': 0.15430021, '泰國': 0.22564119, '籍': 0.15457964, '女子': 0.20561604, '玉': 0.18503447, '婉': 0.2801289, '名字': 0.1893617, 'J': 0.09639844, 'ARU': 0.15340573, 'WAN': 0.1527559, 'W': 0.10614167, 'ONG': 0.17406152, 'P': 0.08019862, 'RAT': 0.14316642, 'OOM': 0.25425714, '涉嫌': 0.16655497, '本': 0.022869216, '案': 0.16948897, '偽': 0.23804182, '造': 0.19358818, '文書': 0.24949726, '部分': 0.041564226, '臺灣': 0.1823859, '臺': 0.17620263, '南': 0.1033398, '地方': 0.021239895, '法院': 0.11220011, '檢察': 0.15207851, '署': 0.0019599125, '官': 0.02064361, '通': 0.08354054, '緝': 0.22676866, '中': 0.01608228, '間': 0.032448836, '無': 0.13647482, '結婚': 0.26993293, '真': 0.09816149, '意': 0.022240553, '張': 0.16134399, '晉': 0.21324842, '源': 0.22151151, '等': 0.021718606, '欲': 0.115392976, '找': 0.12489739, '男子': 0.16522005, '前往': 0.12640253, '辦理': 0.14597866, '假': 0.23169234, '林': 0.12794648, '清水': 0.20588462, '負責': 0.010313436, '介紹': 0.1085398

In [ ]:
lexical_scores1 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
lexical_scores2 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][1])

lexical_scores3 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][0])
lexical_scores4 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][1])

print(f'{lexical_scores1} {lexical_scores2}')
print(f'{lexical_scores3} {lexical_scores4}')

0.36429209308698773 0.1011430734906753
0.09015482128597796 0.25684958447527606


In [ ]:
output_1 = model.encode(cc_tokenized_sample_1, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(cc_tokenized_sample_2, return_dense=True, return_sparse=True, return_colbert_vecs=False)

print(model.convert_id_to_token(output_1['lexical_weights']))

# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))

[{'': 0.05450929, '明': 0.20648107, '知': 0.29140913, '与': 0.28396952, '籍': 0.24085203, '女子': 0.25043726, '名字': 0.19686082, '为': 0.12647548, 'J': 0.13592005, 'ARU': 0.15880142, 'WAN': 0.19843721, 'W': 0.08575899, 'ONG': 0.14353639, 'P': 0.07869031, 'RAT': 0.1598173, 'OOM': 0.27195808, '涉嫌': 0.19210841, '本': 0.017413981, '案': 0.16671188, '伪': 0.2339328, '造': 0.1773647, '文': 0.13739982, '书': 0.20512949, '部分': 0.0564465, '地方': 0.055909, '法院': 0.10825533, '检察': 0.12534922, '官': 0.0072517022, '通': 0.03940476, '缉': 0.24454391, '中': 0.01402327, '间': 0.029456668, '并': 0.03237748, '无': 0.1655582, '结婚': 0.2849247, '真': 0.096296124, '意': 0.13781816, '源': 0.2020495, '等': 0.023157053, '欲': 0.09399369, '找': 0.13135943, '男子': 0.16969621, '前往': 0.11149996, '办理': 0.14429, '假': 0.23981307, '林': 0.15007254, '清水': 0.2241843, '负责': 0.10493385, '介绍': 0.15919706, '予': 0.06341365, '人': 0.03836564, '赚': 0.16588175, '取': 0.01681481, '新': 0.056661643, '台': 0.18780553, '币': 0.15654057, '下': 0.016991924, '同': 0.0254

In [ ]:
lexical_scores1 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
lexical_scores2 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][1])

lexical_scores3 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][0])
lexical_scores4 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][1])

print(f'{lexical_scores1} {lexical_scores2}')
print(f'{lexical_scores3} {lexical_scores4}')

0.5111280533892568 0.1395197610836476
0.1275615413323976 0.29369049098386313


In [ ]:
output_1 = model.encode(sample_3, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(sample_4, return_dense=True, return_sparse=True, return_colbert_vecs=False)

print(model.convert_id_to_token(output_1['lexical_weights']))

# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))

[{'己': 0.24949853, '○○': 0.21181884, '與': 0.092471175, '經營': 0.19112001, '金': 0.17044678, '晉': 0.24103169, '安': 0.20256236, '人力': 0.18579343, '仲介': 0.19294608, '有限公司': 0.15217295, '官': 0.18751574, '威': 0.2310353, '成': 0.21062735, '、': 0.03673291, '怡': 0.21308821, '秀': 0.21219215, '涉': 0.06764519, '違反': 0.19296533, '就業': 0.18222195, '服務': 0.16803238, '法': 0.10056808, '等': 0.024950303, '案件': 0.10885369, ',': 0.024395816, '分別': 0.040344298, '臺灣': 0.19430293, '南投': 0.1477972, '地方': 0.02846387, '法院': 0.11422471, '95': 0.17947875, '年度': 0.089316085, '易': 0.14907512, '字': 0.021837287, '第': 0.048283678, '53': 0.15322298, '號': 0.024714343, '判': 0.1293755, '處': 0.13921583, '有期徒刑': 0.24008499, '10': 0.09736256, '月': 0.024471205, '高等': 0.023227133, '臺': 0.19847867, '中': 0.06706471, '分': 0.1491487, '院': 0.01305303, '以': 0.025113717, '99': 0.15139693, '上': 0.023977656, '94': 0.10927415, '6': 0.07438111, '確定': 0.14778467, '真實': 0.19627538, '姓名': 0.13756905, '年': 0.11792795, '籍': 0.13757485, '不': 0.12

In [ ]:
lexical_scores1 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
lexical_scores2 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][1])
lexical_scores3 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][2])

lexical_scores4 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][0])
lexical_scores5 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][1])
lexical_scores6 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][2])

lexical_scores7 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][0])
lexical_scores8 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][1])
lexical_scores9 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][2])

print(f'{lexical_scores1} {lexical_scores2} {lexical_scores3}')
print(f'{lexical_scores4} {lexical_scores5} {lexical_scores6}')
print(f'{lexical_scores7} {lexical_scores8} {lexical_scores9}')

1.0886423303586525 0.3831455946240112 0.38493005066993646
0.27445293439086527 0.402689978201181 0.37952038703951985
0.3369303013168974 0.35157026874560415 2.7369453321980473


In [ ]:
output_1 = model.encode(tokenized_sample_3, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(tokenized_sample_4, return_dense=True, return_sparse=True, return_colbert_vecs=False)

print(model.convert_id_to_token(output_1['lexical_weights']))

# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))

[{'經營': 0.23106717, '金': 0.22215083, '人力': 0.19755243, '仲介': 0.2430965, '有限公司': 0.18681578, '官': 0.20664075, '威': 0.25679636, '成': 0.21859004, '涉': 0.12265912, '違反': 0.22415353, '': 0.009732299, '就業': 0.20872307, '服務': 0.22841705, '法': 0.1613704, '案件': 0.13216618, '分別': 0.07402485, '臺灣': 0.1886867, '南投': 0.13048801, '地方': 0.020976283, '法院': 0.13424242, '年度': 0.11446829, '易': 0.16214943, '字': 0.05330687, '號': 0.015390031, '判': 0.136271, '處': 0.11005674, '有期徒刑': 0.24844971, '高等': 0.019260101, '臺': 0.13268411, '中': 0.06684701, '院': 0.02059358, '上': 0.044759147, '確定': 0.14192514, '真實': 0.22802193, '姓名': 0.1665453, '籍': 0.1724526, '不': 0.06680078, '詳': 0.15578593, '綽': 0.20556319, '年': 0.12541798, '女子': 0.18218562, '明': 0.1487738, '知': 0.23481369, '表': 0.011119336, '外籍': 0.19408923, '麗': 0.15711048, '娜': 0.24559414, '工作': 0.16217634, '無': 0.17202279, '結婚': 0.23072706, '真': 0.11060034, '意': 0.12605956, '負責': 0.16488542, '居': 0.11713222, '介紹': 0.052577607, '戊': 0.16554372, '擔任': 0.11732267, '

In [ ]:
lexical_scores1 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
lexical_scores2 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][1])
lexical_scores3 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][2])

lexical_scores4 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][0])
lexical_scores5 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][1])
lexical_scores6 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][2])

lexical_scores7 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][0])
lexical_scores8 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][1])
lexical_scores9 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][2])

print(f'{lexical_scores1} {lexical_scores2} {lexical_scores3}')
print(f'{lexical_scores4} {lexical_scores5} {lexical_scores6}')
print(f'{lexical_scores7} {lexical_scores8} {lexical_scores9}')

0.8052082733747739 0.3592659393834765 0.1626352117418719
0.3098407103084355 0.49542216573991027 0.3589866917955078
0.3123540516862704 0.3545336989554926 1.8461087157593283


In [ ]:
output_1 = model.encode(cc_tokenized_sample_3, return_dense=True, return_sparse=True, return_colbert_vecs=False)
output_2 = model.encode(cc_tokenized_sample_4, return_dense=True, return_sparse=True, return_colbert_vecs=False)

print(model.convert_id_to_token(output_1['lexical_weights']))

# compute the scores via lexical mathcing
lexical_scores = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
print(lexical_scores)

print(model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_1['lexical_weights'][1]))

[{'与': 0.19463667, '经营': 0.25164324, '金': 0.20276935, '人力': 0.18788807, '仲介': 0.221133, '有限公司': 0.17720011, '官': 0.18080811, '威': 0.2193365, '成': 0.17285642, '涉': 0.13828039, '违反': 0.23833296, '就业': 0.22129744, '服务': 0.21554843, '法': 0.17030583, '': 0.03417991, '案件': 0.16019544, '分别': 0.059859477, '经': 0.021375611, '南投': 0.16903314, '地方': 0.022856288, '法院': 0.16138215, '年度': 0.117794536, '易': 0.15559456, '字': 0.03466467, '号': 0.064973645, '判': 0.17420346, '处': 0.15890427, '有期徒刑': 0.27331036, '高等': 0.06475803, '台中': 0.13897598, '分': 0.057719506, '院': 0.027844079, '上': 0.021218225, '确定': 0.19514719, '真实': 0.24822325, '姓名': 0.19814737, '年': 0.15161157, '籍': 0.15664369, '不': 0.109375834, '详': 0.14572164, '绰': 0.15270476, '女子': 0.1964624, '明': 0.1749071, '知': 0.21874839, '附': 0.04485316, '表': 0.021830041, '所示': 0.028266847, '外籍': 0.24199468, '人': 0.065838166, '仅': 0.084658995, '为': 0.021461494, '来': 0.14258012, '台': 0.23128657, '工作': 0.15168023, '无': 0.19748618, '结婚': 0.24578516, '真': 0.138

In [ ]:
lexical_scores1 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
lexical_scores2 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][1])
lexical_scores3 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][2])

lexical_scores4 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][0])
lexical_scores5 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][1])
lexical_scores6 = model.compute_lexical_matching_score(output_1['lexical_weights'][1], output_2['lexical_weights'][2])

lexical_scores7 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][0])
lexical_scores8 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][1])
lexical_scores9 = model.compute_lexical_matching_score(output_1['lexical_weights'][2], output_2['lexical_weights'][2])

print(f'{lexical_scores1} {lexical_scores2} {lexical_scores3}')
print(f'{lexical_scores4} {lexical_scores5} {lexical_scores6}')
print(f'{lexical_scores7} {lexical_scores8} {lexical_scores9}')

1.0643429195642966 0.2776009238077677 0.2348859452904435
0.29742844013526337 0.4435920581781829 0.3631424810446333
0.39091309281866415 0.33642640582365857 1.7690151336173585


## colbert

In [99]:
output_1 = model.encode(sword_1, return_dense=True, return_sparse=True, return_colbert_vecs=True)
output_2 = model.encode(sword_2, return_dense=True, return_sparse=True, return_colbert_vecs=True)

print(model.colbert_score(output_1['colbert_vecs'][0], output_2['colbert_vecs'][0]))
print(model.colbert_score(output_1['colbert_vecs'][0], output_2['colbert_vecs'][1]))
print(model.colbert_score(output_1['colbert_vecs'][1], output_2['colbert_vecs'][0]))
print(model.colbert_score(output_1['colbert_vecs'][1], output_2['colbert_vecs'][1]))

tensor(0.8653)
tensor(0.5597)
tensor(0.5299)
tensor(0.7482)


In [148]:
len(sword_1)

2

In [150]:
print(output_1['colbert_vecs'][0].shape)
print(output_1['colbert_vecs'][1].shape)

(3, 1024)
(3, 1024)


In [134]:
train_embeddings_colbert = model.encode(tokenized_train_summaries[:10], return_dense=True, return_sparse=True, return_colbert_vecs=True)['colbert_vecs']

In [135]:
test_embeddings_colbert = model.encode(tokenized_test_summaries[:2], return_dense=True, return_sparse=True, return_colbert_vecs=True)['colbert_vecs']

In [ ]:
np.savetxt('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/train_embeddings_colbert.txt', train_embeddings_colbert)
np.savetxt('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/summary/test_embeddings_colbert.txt', test_embeddings_colbert)

In [160]:
k = 3
knn_model_colbert = NearestNeighbors(n_neighbors=k, metric=model.colbert_score)

In [145]:
print(len(tokenized_train_summaries[0]))
print(len(tokenized_train_summaries[1]))
print(len(tokenized_train_summaries[2]))

248
363
588


In [142]:
print(train_embeddings_colbert[0].shape)
print(train_embeddings_colbert[1].shape)
print(train_embeddings_colbert[2].shape)

(194, 1024)
(315, 1024)
(441, 1024)


In [152]:
max_length = max(len(vec) for vec in train_embeddings_colbert)
max_length

546

In [155]:
vec_1024 = [0]*1024

In [157]:
padded_train_embeddings_colbert = []
for embedding in train_embeddings_colbert:
  zero_embedding = np.zeros((max_length, 1024))
  for i, vec in enumerate(embedding):
    zero_embedding[i] = vec
  padded_train_embeddings_colbert.append(zero_embedding)

In [158]:
print(padded_train_embeddings_colbert[0].shape)
print(padded_train_embeddings_colbert[1].shape)
print(padded_train_embeddings_colbert[2].shape)

(546, 1024)
(546, 1024)
(546, 1024)


In [161]:
knn_model_colbert.fit(padded_train_embeddings_colbert)

ValueError: Found array with dim 3. NearestNeighbors expected <= 2.

In [114]:
# 將模型保存到文件
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/knn_model_colbert.pkl', 'wb') as f:
    pickle.dump(knn_model_colbert, f)

# 從文件加載模型
with open('/content/drive/MyDrive/senior/LAB/LawCaseRetreiver/sentence_predict/knn_model_colbert.pkl', 'rb') as f:
    knn_model_colbert_loaded = pickle.load(f)

In [125]:
len(train_embeddings_colbert['colbert_vecs'])

10

In [132]:
test_embeddings_colbert['dense_vecs']

array([[-0.01875211,  0.02883617,  0.00412132, ..., -0.01654896,
        -0.05432812,  0.00466428],
       [-0.03423084,  0.03943255, -0.06090587, ...,  0.01285641,
        -0.03339079,  0.00087781]], dtype=float32)

In [133]:
test_embeddings_colbert['colbert_vecs']

[array([[-0.01347857,  0.05767947,  0.00216896, ...,  0.04237657,
          0.03365747, -0.0112196 ],
        [ 0.00700776,  0.03143869,  0.01361316, ...,  0.04598797,
          0.00146442, -0.00404962],
        [-0.02111956,  0.05814525,  0.00954774, ..., -0.01847027,
          0.01094045, -0.00482355],
        ...,
        [-0.01889718,  0.07741033, -0.00618311, ...,  0.04854978,
         -0.00848097, -0.03542492],
        [-0.01817219,  0.04910505, -0.01528996, ...,  0.00565172,
         -0.02460399, -0.00530999],
        [ 0.01880972,  0.01375304, -0.00312364, ...,  0.02246102,
          0.02100301,  0.01338852]], dtype=float32),
 array([[-0.03183267,  0.03221238, -0.01905505, ...,  0.01542058,
          0.0207462 , -0.01091856],
        [ 0.02097862,  0.03592551, -0.03860803, ...,  0.03772056,
          0.01515677,  0.0106566 ],
        [-0.04141861,  0.0314973 , -0.0254971 , ...,  0.02118472,
          0.01958897, -0.01210931],
        ...,
        [-0.0192519 ,  0.0606344 , -0.0

In [121]:
len(test_embeddings_colbert['colbert_vecs'][0])

231

In [118]:
distances, indices = knn_model_colbert_loaded.kneighbors(test_embeddings_colbert['colbert_vecs'][0][0].reshape(-1, 1))

ValueError: X has 1 features, but NearestNeighbors is expecting 1024 features as input.

# 平均

In [ ]:
avg_sentence = sum(train_sentences)/len(train_sentences)

In [ ]:
avg_sentence

187.724

In [ ]:
avg_sentences = [avg_sentence]*len(test_sentences)

In [ ]:
mse_linear = mean_squared_error(test_sentences, avg_sentences)
rmse_linear = mean_squared_error(test_sentences, avg_sentences, squared=False)
mae_linear = mean_absolute_error(test_sentences, avg_sentences)

In [ ]:
print('\n==================================================')
print("Linear Regression - MSE:", mse_linear)
print("Linear Regression - RMSE:", rmse_linear)
print("Linear Regression - MAE:", mae_linear)


Linear Regression - MSE: 16325.817136000003
Linear Regression - RMSE: 127.77252105206347
Linear Regression - MAE: 89.10199999999999
